1) calcula as matrizes Sb e Sw a partir dos dados de treinamento.(check)

2) calcula os autovetores e autovalores da matriz "(Sw^-1)*Sb(check)

3) verifica os "m" maiores autovalores, e seleciona os "m" autovetores correspondentes para montar a matriz W (dimensao p x m, onde p é o número de atributos).

4) faz a transformação dos dados de treinamento: Xtr_t = W' * Xtr (onde Xtr_t é a matriz transformada, W' é o transposto da matriz de transformação, e X é a matriz de dados de treinamento, de dimensão p x Ntr)

5) Treina o classificador com essa matriz transformada.

6) transforma os dados de teste: Xts_t = W' * Xts

7) Classificar os dados de teste transformados Xts_t


In [47]:
import pandas as pd
import numpy as np
import funcoes_importantes as fi
fp = fi.funcoes_main()
derma_dados = pd.read_csv('https://raw.githubusercontent.com/SamuelHericles/Algoritmos_de_classificao_baysianos/master/01.Dados/dema_dados.csv')
derma_dados.sort_values('c35',inplace=True)
derma_dados.reset_index(drop=True,inplace=True)
derma_dados

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c26,c27,c28,c29,c30,c31,c32,c33,c34,c35
0,1,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,60,1
1,2,2,2,0,0,0,0,0,3,2,...,2,0,0,0,0,0,3,0,30,1
2,3,2,3,0,0,0,0,0,2,2,...,2,0,0,0,0,0,2,0,33,1
3,2,3,2,0,0,0,0,0,2,0,...,0,0,0,0,0,0,2,0,27,1
4,3,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,56,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2,2,1,1,0,0,2,0,2,0,...,0,0,2,0,1,2,2,0,10,6
354,2,2,1,0,0,0,2,0,2,0,...,0,0,0,0,1,1,1,0,13,6
355,2,1,1,2,0,0,3,0,1,2,...,0,0,0,0,1,2,1,0,8,6
356,3,2,0,1,0,0,2,0,1,0,...,0,0,3,0,1,2,2,0,12,6


In [48]:
def treino_teste_norm(derma_dados):
    X = pd.DataFrame({})
    y = pd.DataFrame({})

    index_teste_classe = sorted(np.random.choice(derma_dados.index.values,int(derma_dados.shape[0]*0.3)))
    X_classe = derma_dados.iloc[[i for i in derma_dados.index if i not in index_teste_classe],:]
    y_classe = derma_dados.iloc[index_teste_classe,:]

    X = X.append(X_classe,ignore_index=True)
    y = y.append(y_classe,ignore_index=True)

    X_norm,y_norm = norm_z_score(X, y)
    return X_norm,y_norm

In [49]:
def norm_z_score(base_trenio, base_teste):
    media         = base_trenio[base_trenio.columns[:-1]].mean()
    desvio_padrao = base_trenio[base_trenio.columns[:-1]].std()
    base_trenio[base_trenio.columns[:-1]] = base_trenio[base_trenio.columns[:-1]]-media/desvio_padrao
    base_teste[base_teste.columns[:-1]]   = base_teste[base_teste.columns[:-1]]-media/desvio_padrao
    return base_trenio,base_teste

def matriz_covariancia(base):
    df = pd.DataFrame(index=[base.columns[i] for i in range(base.shape[1])],columns=base.columns[:-1])
    for i,count_i in zip(base.columns[:-1],[i for i in range(base.shape[1])]):
        for j,count_j in zip(base.columns[:-1],[i for i in range(base.shape[1])]):
            df.iloc[count_i,count_j] = (sum(base[str(i)]*base[str(j)])- 
                                        (sum(base[str(i)])*sum(base[str(j)]))/base.shape[0])/base.shape[0] 
    return df.iloc[:-1]

def matriz_inversa(matriz_covariancia):
     return pd.DataFrame(np.linalg.inv(np.matrix(matriz_covariancia.values, dtype='float')))

def vetor_medio(base):
    vetor_medio = []
    for i,j in zip(base.columns[:-1],[i for i in range(base.shape[1])]):
        vetor_medio.append(base[str(i)].mean())
    df = pd.DataFrame(data=vetor_medio,index=base.columns[:-1])
    return df.T

def vetor_medio_p_classe(base):
    vt_medios = pd.DataFrame(index=base.columns[:-1])
    for i in sorted(base[base.columns[-1]].unique()):
        vt_medios['classe'+str(i)] = vetor_medio(base.query('c35=='+str(i))).T
    return vt_medios.T
    
def get_acc(y_pred,y_true):
    return round(sum(y_pred==y_true)/y_true.shape[0],4)

def CDA(X): #CDA
    
    # 1 - pegar as matrizes de covariancia
    mcov_1 = matriz_covariancia(X.query('c35==1'))
    mcov_2 = matriz_covariancia(X.query('c35==2'))
    mcov_3 = matriz_covariancia(X.query('c35==3'))
    mcov_4 = matriz_covariancia(X.query('c35==4'))
    mcov_5 = matriz_covariancia(X.query('c35==5'))
    mcov_6 = matriz_covariancia(X.query('c35==6'))
    
    # 2 - matriz de covairanci Sw
    Sw = mcov_1+mcov_2+mcov_3+mcov_4+mcov_5+mcov_6
    
    # 3 - Calcular o vetor médio de cada classse
    vt_1 = vetor_medio(X.query('c35==1'))
    vt_2 = vetor_medio(X.query('c35==2'))
    vt_3 = vetor_medio(X.query('c35==3'))
    vt_4 = vetor_medio(X.query('c35==4'))
    vt_5 = vetor_medio(X.query('c35==5'))
    vt_6 = vetor_medio(X.query('c35==6'))
    
    # 4 - Calcular a media dos vetores médios
    vt = (vt_1+vt_2+vt_3+vt_4+vt_5+vt_6)/6
    
    # 5 - Calcular a projeção dos vetores médios
    sb1 = pd.DataFrame(X.query('c35==1').shape[0]*((vt_1 - vt).values*(vt_1 - vt).T.values))
    sb2 = pd.DataFrame(X.query('c35==2').shape[0]*((vt_2 - vt).values*(vt_2 - vt).T.values))
    sb3 = pd.DataFrame(X.query('c35==3').shape[0]*((vt_3 - vt).values*(vt_3 - vt).T.values))
    sb4 = pd.DataFrame(X.query('c35==4').shape[0]*((vt_4 - vt).values*(vt_4 - vt).T.values))
    sb5 = pd.DataFrame(X.query('c35==5').shape[0]*((vt_5 - vt).values*(vt_5 - vt).T.values))
    sb6 = pd.DataFrame(X.query('c35==6').shape[0]*((vt_6 - vt).values*(vt_6 - vt).T.values))

    # 6 - Somar os  Sb's
    Sb = sb1+sb2+sb3+sb4+sb5+sb6
    
    m_aa_av = matriz_inversa(Sw)*Sb
    
    a_valores,a_vetores = np.linalg.eig(m_aa_av)
    df_a_valores = pd.DataFrame(a_valores)
    df_a_vetores = pd.DataFrame(a_vetores)
    
    W = pd.DataFrame({})
    for i in df_a_valores.sort_values(by=0,ascending=True)[:5].index.values:
        W = W.append(df_a_vetores.iloc[i],ignore_index=True)

    #return pd.DataFrame(W.values @ X[X.columns[:-1]].T.values)
    return W

def correcao_lambda(matriz_de_covariancia):
     return matriz_de_covariancia + np.identity(matriz_de_covariancia.shape[0], dtype=float)*0.01
    
def teste_elem_lda(y,X_medio,mcov_inv):
    y = y.values.reshape((1,-1))
    X_medio = X_medio.values.reshape((1,-1))
    m_linha   = (y-X_medio)    
    m_coluna  = (y-X_medio).T    
    resultado = (m_linha @ mcov_inv) @ m_coluna
    return resultado[0].values[0].round(2)

def teste_elem(X,X_medio,mcov_inv,determinante,probabilidade):
#     print(X)
#     print(X_medio)
    det_log  = np.log(abs(determinante))
    m_linha  = (X-X_medio).values
    m_coluna = (X-X_medio).T.values
    log_prob = np.log(probabilidade)
#     print(m_linha)
#     print(m_coluna)
#     print('mcov',mcov_inv.shape)
#     print(det_log)
    
    resultado = det_log + (m_linha @ mcov_inv) @ (m_coluna) - 2*log_prob
    return resultado[0][0]

In [50]:
X_norm,y_norm = treino_teste_norm(derma_dados)

W = CDA(X_norm)


In [51]:
colunas = {0:'c1',1:'c2',2:'c3',3:'c4',4:'c5'}

Xtr_t = pd.DataFrame(W.values @  X_norm[X_norm.columns[:-1]].T.values).T
Xtr_t['c35'] = X_norm['c35']
Xtr_t.rename(columns=colunas,inplace=True)

Xts_t = pd.DataFrame(W.values @  y_norm[y_norm.columns[:-1]].T.values).T
Xts_t['c35'] = y_norm['c35']
Xts_t.rename(columns=colunas,inplace=True)


In [52]:
Xtr_t

,c1,c2,c3,c4,c5,c35
0,-0.540795,-0.845267,0.213609,0.116961,0.084778,1
1,0.026620,-1.713985,-0.204161,-0.266098,0.975340,1
2,-0.378874,-2.438158,-0.329982,-0.393679,1.001035,1
3,0.555224,-0.894691,1.027832,0.384286,-0.086601,1
4,-0.319613,-1.982321,-0.781426,-0.740021,0.058359,1
...,...,...,...,...,...,...
259,-0.429206,1.397056,-0.796809,0.322551,0.479011,6
260,-0.127680,1.017320,0.188617,0.573077,-0.148855,6
261,-0.082486,0.918894,0.187282,0.702792,-0.159036,6
262,-0.104938,-0.230571,-0.746629,0.399274,-0.871473,6


In [53]:
Xts_t

,c1,c2,c3,c4,c5,c35
0,-0.526987,-1.453187,-0.319121,-0.294500,0.586051,1
1,-0.346619,-1.470238,-0.187298,0.073484,1.150871,1
2,-0.948664,-0.928206,-0.179935,-0.409190,-0.167631,1
3,-1.106721,-0.852397,-0.173606,-0.411991,0.002208,1
4,-1.563287,0.714687,-1.324610,-0.928672,0.775436,1
...,...,...,...,...,...,...
102,-0.558371,-0.289721,0.143941,0.931403,-1.441661,6
103,-0.544189,1.401910,-0.748507,0.807615,0.048864,6
104,-0.544189,1.401910,-0.748507,0.807615,0.048864,6
105,-0.160876,2.232082,0.281975,0.790922,-0.178897,6


In [54]:
def qda(X,y):
    # 1- calcula a probilidade a priori[1x1]
    probs_p = fp.prob_a_priori(X)

    # 2- vetor médio por classe[1x34]
    vt_medio = fp.vetor_medio_p_classe(X)

    
    # 3- matriz covariancia por classe[34x34]
    mcov_1 = fp.matriz_covariancia(X.query('c35==1'))
    mcov_2 = fp.matriz_covariancia(X.query('c35==2'))
    mcov_3 = fp.matriz_covariancia(X.query('c35==3'))
    mcov_4 = fp.matriz_covariancia(X.query('c35==4'))
    mcov_5 = fp.matriz_covariancia(X.query('c35==5'))
    mcov_6 = fp.matriz_covariancia(X.query('c35==6'))
    a = pd.DataFrame([mcov_1.values,mcov_2.values,mcov_3.values,mcov_4.values,mcov_5.values,mcov_6.values])
    
    # 4- Fazer a correção lamdba A = A + xI na matriz de covariancia
    mcov_1_lambda = fp.correcao_lambda(mcov_1)
    mcov_2_lambda = fp.correcao_lambda(mcov_2)
    mcov_3_lambda = fp.correcao_lambda(mcov_3)
    mcov_4_lambda = fp.correcao_lambda(mcov_4)
    mcov_5_lambda = fp.correcao_lambda(mcov_5)
    mcov_6_lambda = fp.correcao_lambda(mcov_6)

    # 5- Calcular a inversa da matriz de covairância regulada lambda
    mcov_1_lambda_inv = fp.matriz_inversa(mcov_1_lambda)
    mcov_2_lambda_inv = fp.matriz_inversa(mcov_2_lambda)
    mcov_3_lambda_inv = fp.matriz_inversa(mcov_3_lambda)
    mcov_4_lambda_inv = fp.matriz_inversa(mcov_4_lambda)
    mcov_5_lambda_inv = fp.matriz_inversa(mcov_5_lambda)
    mcov_6_lambda_inv = fp.matriz_inversa(mcov_6_lambda)

    # 6- Calcular o determinante de cada matriz de covariância regulada
    det_1 = fp.determinante_matriz_covariancia(mcov_1_lambda)
    det_2 = fp.determinante_matriz_covariancia(mcov_2_lambda)
    det_3 = fp.determinante_matriz_covariancia(mcov_3_lambda)
    det_4 = fp.determinante_matriz_covariancia(mcov_4_lambda)
    det_5 = fp.determinante_matriz_covariancia(mcov_5_lambda)
    det_6 = fp.determinante_matriz_covariancia(mcov_6_lambda)


    # Teste, pegar agora um elementro da base de teste e aplicar
    # min: ln|det(mcov_classe_i)| + 
    #       (elem_teste-vetor_medio_classe_i).T*mcov_classe_i_inv*(elem_teste-vetor_md_classe_i)
    #       - 2*ln(P_priori(classe_i)) 

    # elem_teste é X da questão, literalmente
    # o que o min retorna?
    # min retorna um valor 1,2,3,4,5,6 sendo este a classe

    aux = []
    for i in y.index.values:
        valores = []
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[0],mcov_1_lambda_inv,det_1,probs_p[0]))    
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[1],mcov_2_lambda_inv,det_2,probs_p[1]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[2],mcov_3_lambda_inv,det_3,probs_p[2]))    
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[3],mcov_4_lambda_inv,det_4,probs_p[3]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[4],mcov_5_lambda_inv,det_5,probs_p[4]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio.iloc[5],mcov_6_lambda_inv,det_6,probs_p[5]))
        aux.append(valores.index(min(valores))+1)
    print('Acrurácia de ',fp.get_acc(aux,y['c35'])*100,'%')
    return fp.get_acc(aux,y['c35'])

In [55]:
def lda(X,y):
    # 1- vetor médio por classe[1x34]
    vt_medio_1 = fp.vetor_medio(X.query('c35==1'))
    vt_medio_2 = fp.vetor_medio(X.query('c35==2'))
    vt_medio_3 = fp.vetor_medio(X.query('c35==3'))
    vt_medio_4 = fp.vetor_medio(X.query('c35==4'))
    vt_medio_5 = fp.vetor_medio(X.query('c35==5'))
    vt_medio_6 = fp.vetor_medio(X.query('c35==6'))


    # 2- mcov_medio ou agregaddo é [34x34] somar todas as mcov de n classes e fazer a média mcov/n classes [34x34]
    mcov_1 = fp.matriz_covariancia(X.query('c35==1'))
    mcov_2 = fp.matriz_covariancia(X.query('c35==2'))
    mcov_3 = fp.matriz_covariancia(X.query('c35==3'))
    mcov_4 = fp.matriz_covariancia(X.query('c35==4'))
    mcov_5 = fp.matriz_covariancia(X.query('c35==5'))
    mcov_6 = fp.matriz_covariancia(X.query('c35==6'))
    mcov_medio = mcov_1+mcov_2+mcov_3+mcov_4+mcov_5+mcov_6
    mcov_medio = mcov_medio/6

    # 3- Fazer a correção lamdba A = A + xI na matriz de covariancia
    mcov_medio_lambda = mcov_medio + np.identity(mcov_medio.shape[0], dtype=float)*0.01

    # 5- Calcular a inversa da matriz de covairância regulada lambda
    mcov_medio_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_medio_lambda.values, dtype='float')))
    
    # 4- Teste, pegar agora um elementro da base de teste e aplicar
    # min: (elem_teste-vetor_medio_classe_i).T*mcov_classe_i_inv_media(agredada)*(elem_teste-vetor_md_classe_i)
    aux = []
    for i in y.index.values:
        valores = []
        valores.append(fp.teste_elem_lda(y.iloc[i,:-1],vt_medio_1,mcov_medio_lambda_inv_d))    
        valores.append(fp.teste_elem_lda(y.iloc[i,:-1],vt_medio_2,mcov_medio_lambda_inv_d))
        valores.append(fp.teste_elem_lda(y.iloc[i,:-1],vt_medio_3,mcov_medio_lambda_inv_d))    
        valores.append(fp.teste_elem_lda(y.iloc[i,:-1],vt_medio_4,mcov_medio_lambda_inv_d))
        valores.append(fp.teste_elem_lda(y.iloc[i,:-1],vt_medio_5,mcov_medio_lambda_inv_d))
        valores.append(fp.teste_elem_lda(y.iloc[i,:-1],vt_medio_6,mcov_medio_lambda_inv_d))
        aux.append(valores.index(min(valores))+1)
    print('Acrurácia de ',fp.get_acc(aux,y['c35'])*100,'%')
    return fp.get_acc(aux,y['c35'])

In [56]:
def nayves_bayes(X,y):
    # 1- calcula a probilidade a priori[1x1]
    probs_p = fp.prob_a_priori(X)

    # 2- vetor médio por classe[1x34]
    vt_medio_1 = fp.vetor_medio(X.query('c35==1'))
    vt_medio_2 = fp.vetor_medio(X.query('c35==2'))
    vt_medio_3 = fp.vetor_medio(X.query('c35==3'))
    vt_medio_4 = fp.vetor_medio(X.query('c35==4'))
    vt_medio_5 = fp.vetor_medio(X.query('c35==5'))
    vt_medio_6 = fp.vetor_medio(X.query('c35==6'))
    
    mcov_1 = fp.matriz_covariancia(X.query('c35==1'))
    mcov_2 = fp.matriz_covariancia(X.query('c35==2'))
    mcov_3 = fp.matriz_covariancia(X.query('c35==3'))
    mcov_4 = fp.matriz_covariancia(X.query('c35==4'))
    mcov_5 = fp.matriz_covariancia(X.query('c35==5'))
    mcov_6 = fp.matriz_covariancia(X.query('c35==6'))

    # 3- matriz covariancia por classe[34x34] sabendo que são descorrelacionados
    mcov_1_descorrelacionada = fp.limpar_covariancia(mcov_1)
    mcov_2_descorrelacionada = fp.limpar_covariancia(mcov_2)
    mcov_3_descorrelacionada = fp.limpar_covariancia(mcov_3)
    mcov_4_descorrelacionada = fp.limpar_covariancia(mcov_4)
    mcov_5_descorrelacionada = fp.limpar_covariancia(mcov_5)
    mcov_6_descorrelacionada = fp.limpar_covariancia(mcov_6)

    # 4- Fazer a correção lamdba A = A + xI na matriz de covariancia
    mcov_1_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_2_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_3_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_4_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_5_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_6_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01

    # 5- Calcular a inversa da matriz de covairância regulada lambda
    mcov_1_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_1_lambda_d.values, dtype='float')))
    mcov_2_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_2_lambda_d.values, dtype='float')))
    mcov_3_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_3_lambda_d.values, dtype='float')))
    mcov_4_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_4_lambda_d.values, dtype='float')))
    mcov_5_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_5_lambda_d.values, dtype='float')))
    mcov_6_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_6_lambda_d.values, dtype='float')))
    
    # 6- Calcular o determinante de cada matriz de covariância regulada
    det_1 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_2 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_3 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_4 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_5 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_6 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    
    
    # 5- Teste, pegar agora um elementro da base de teste e aplicar
    # min: ln|det(mcov_classe_i)| + 
    #       (elem_teste-vetor_medio_classe_i).T*mcov_classe_i_inv*(elem_teste-vetor_md_classe_i)
    #       - 2*ln(P_priori(classe_i)) 
    aux = []
    for i in y.index.values:
        valores = []
        valores.append(teste_elem(y.iloc[i,:-1],vt_medio_1,mcov_1_lambda_inv_d,det_1,probs_p[0]))    
        valores.append(teste_elem(y.iloc[i,:-1],vt_medio_2,mcov_2_lambda_inv_d,det_2,probs_p[1]))
        valores.append(teste_elem(y.iloc[i,:-1],vt_medio_3,mcov_3_lambda_inv_d,det_3,probs_p[2]))    
        valores.append(teste_elem(y.iloc[i,:-1],vt_medio_4,mcov_4_lambda_inv_d,det_4,probs_p[3]))
        valores.append(teste_elem(y.iloc[i,:-1],vt_medio_5,mcov_5_lambda_inv_d,det_5,probs_p[4]))
        valores.append(teste_elem(y.iloc[i,:-1],vt_medio_6,mcov_6_lambda_inv_d,det_6,probs_p[5]))
        aux.append(valores.index(min(valores))+1)
    print('Acrurácia de ',fp.get_acc(aux,y['c35'])*100,'%')
    return fp.get_acc(aux,y['c35'])

In [57]:
Xtr_t_norm,Xts_t_norm = norm_z_score(Xtr_t, Xts_t)
qda(Xtr_t,Xts_t)

Acrurácia de  85.98 %


0.8598

In [58]:
Xtr_t_norm,Xts_t_norm = norm_z_score(Xtr_t, Xts_t)
lda(Xtr_t,Xts_t)


Acrurácia de  77.57 %


0.7757

In [59]:
Xtr_t_norm,Xts_t_norm = norm_z_score(Xtr_t, Xts_t)
nayves_bayes(Xtr_t,Xts_t)

Acrurácia de  71.03 %


0.7103